In [ ]:
import torch
import numpy as np
from PIL import Image, ImageDraw
import d3dshot
import time
import pydirectinput
from pynput.mouse import Button, Controller as MouseController
import pynput
import math
import keyboard

In [ ]:
torch.backends.cudnn.benchmark=True

In [ ]:
#Controlling Mouse: ctypes > win32api > pynput
#Controlling Keyboard: ctypes > pydirectinput != pyautogui

#fire directinputs: .287
#fire ctypes: .057
#(Faster by 5x)

#mouse pynput: .104
#mouse win32api: .016
#mouse ctypes: .008
#(Faster by 13x)

#model cpu: .264
#model gpu: 1.554
#model time is dependent on game stress
#.012 seconds model 5s

In [ ]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model = model.cuda()
from utils.plots import color_list

In [ ]:
import ctypes
SendInput = ctypes.windll.user32.SendInput

W = 0x11
A = 0x1E
S = 0x1F
D = 0x20
L = 0x26

# C struct redefinitions 
PUL = ctypes.POINTER(ctypes.c_ulong)
class KeyBdInput(ctypes.Structure):
    _fields_ = [("wVk", ctypes.c_ushort),
                ("wScan", ctypes.c_ushort),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class HardwareInput(ctypes.Structure):
    _fields_ = [("uMsg", ctypes.c_ulong),
                ("wParamL", ctypes.c_short),
                ("wParamH", ctypes.c_ushort)]

class MouseInput(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time",ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class Input_I(ctypes.Union):
    _fields_ = [("ki", KeyBdInput),
                 ("mi", MouseInput),
                 ("hi", HardwareInput)]

class Input(ctypes.Structure):
    _fields_ = [("type", ctypes.c_ulong),
                ("ii", Input_I)]

# Actuals Functions
def PressKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

def ReleaseKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008 | 0x0002, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

if __name__ == '__main__':
    PressKey(0x11)
    time.sleep(1)
    ReleaseKey(0x11)
    time.sleep(1)

In [ ]:
colors = color_list()
def draw_image(img,pred,colors):
    img = Image.fromarray(img.astype(np.uint8)) if isinstance(img, np.ndarray) else img  # from np
    for *box, conf, cls in pred:
        ImageDraw.Draw(img).rectangle(box, width=4, outline=colors[int(cls) % 10])  # plot    
    return img

In [ ]:
#from pynput.keyboard import Key, Controller as KeyboardController

#keyboard = KeyboardController()
mouse = MouseController()

def unpause_game():
    mouse.move(-10000,10000)
    mouse.move(800,-10)
    time.sleep(1)
    mouse.click(Button.left,1)
    time.sleep(1)
    mouse.click(Button.left,1)

In [ ]:
import ctypes
def click(x,y):
    ctypes.windll.user32.SetCursorPos(x, y)
    ctypes.windll.user32.mouse_event(2, 0, 0, 0,0) # left down
    ctypes.windll.user32.mouse_event(4, 0, 0, 0,0) # left up

In [ ]:
#dir(ctypes.windll.user32)
import win32gui

In [ ]:
def aim_func(num):
    return int(10*math.tanh(.02*num))#*.95*math.log(abs(num)+.001)

In [ ]:
def quad(num,a,b,c):
    return (-b + math.sqrt(b**2 -4*a*(c-num)))/(2*a)

In [ ]:
math.sqrt(1440*1440 + 2560*2560)

In [ ]:
def aim_mouse(pred):
    #get closest person
    closest_dis = 3000
    closest_x = 1280
    closest_y = 720
    for *box, conf, cls in pred:
        #get closest box
        if(int(cls)==0):
            temp1 = time.time()
            x = (box[0] + box[2])/2
            y = (box[3] - box[1])*.25 + box[1]
            #y = (box[1] + box[3])/2
            dis = math.sqrt(x**2 + y**2)
            if(y>1300 or dis>closest_dis):# or (x>1350)):
                continue
            else:
                closest_dis = dis
                closest_x = x
                closest_y = y
    if(closest_dis==3000):
        return 0
                
    x = closest_x
    y = closest_y
        
    #calculate x
    true_dif = abs(1280-x)+.001
    pred_dif = quad(true_dif,.000880357,.874107,12.375)
    chos_dif = min(true_dif,pred_dif)
    new_x = 1280 - ((1280-x)/true_dif)*chos_dif
    new_x = int(new_x)

    #calculate y
    new_y = int(y)
    
    click(new_x,new_y)
    
    #center: 1280, 720
    #print("dx {} detected pos: {} selected pos: {}".format((new_x,new_y),(x,y),(new_x+x,new_y+y)))
    #print("{},{} : {} aim inside for loop".format(x,y,temp2-temp1))

In [ ]:
def fire():
    PressKey(L)
    ReleaseKey(L)

In [ ]:
#unpause_game()
unpause_game()
time.sleep(1.5)
mouse.move(1280,720)
time.sleep(5)

#time.sleep(1.5)

In [ ]:
#does this have to be done in game?
d = d3dshot.create()

In [ ]:
from ctypes import Structure, windll, c_uint, sizeof, byref
import cv2
#windll.user32.mouse_event(1, 500, 0, 0, 0)  This works when mouse_raw_input = True

In [ ]:
#unpause_game()
#call_car()
time.sleep(1)
frames = 1
frame = -1
init_img = 0
init_pred = 0
img_list = []
while(True):
    frame+=1
    #for frame in range(frames):
    if keyboard.is_pressed('l'):
        print("Quit")
        break
        
    a = time.time()
    img = d.screenshot()

    #img = img.resize((512,288))
    b = time.time()
    result = model([img])
    
    c = time.time()
    imgs = result.imgs[0]
    pred = result.pred[0]
    xyxy = result.xyxy
    
    if(frame%4==0):
        aim_mouse(result.pred[0])
    
    if(frame == 0):
        #aim_mouse(result.pred[0])
        init_img = img
        init_pred = pred
    
    e = time.time()
    #print("---- frame {} FPS {} ----".format(frame,1/(e-a)))
    #print("{} seconds screenshot:".format(b-a))
    #print("{} seconds model".format(c-b))
    #print("{} seconds mouse".format(e-c)
    #print("{} seconds since detection".format(e-a))

In [ ]:
img

In [ ]:
draw_image(init_img,init_pred,colors)

In [ ]:
draw_image(img,pred,colors)

In [ ]:
result.print()

In [ ]:
# people = 0
# cars = 4
# trucks = 9
# traffic lights = 2

In [ ]:
img

In [ ]:
'''import cv2
import numpy as np
import glob

img_array = []
for filename in glob.glob('model_imgs/*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()'''